In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
from datawand.parametrization import ParamHelper

In [ ]:
ph = ParamHelper("../pipelines/PopularityModelScores.json",sys.argv)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re, math

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import sys

sys.path.insert(0,'../python/')
import correlation.correlation_utils as cu
import popularity_model.popularity_model as pm

# Choose dataset

In [ ]:
experiment_folder = ph.get("experiment_folder")

In [ ]:
dataset_id = ph.get("dataset_id")

In [ ]:
dataset_stat_file = "%s/centrality_data/%s_results.csv" % (experiment_folder,dataset_id)
stat_df = pd.read_csv(dataset_stat_file, sep=" ")

#### extract number of users in data

In [ ]:
print stat_df.columns[2]

total_num_matcher = re.match(r'.*\(total=(\d+?)\)', stat_df.columns[2], re.M|re.I)
if not total_num_matcher:
    raise RuntimeError("Column name does NOT match the regex!")

#### rename a column

In [ ]:
cols = list(stat_df.columns)
cols[2] = "fraction_of_active_nodes"
stat_df.columns = cols

In [ ]:
stat_df.head()

#### Kendall's Tau is computation intensive: so only a small sample is taken

In [ ]:
num_of_users = 5000 #int(total_num_matcher.group(1))
num_of_days = len(stat_df)-1

In [ ]:
p = list(stat_df["fraction_of_active_nodes"])[:num_of_days]
p_overlap = list(stat_df["fraction_of_users_in_2day_intersections"])[:num_of_days]

# Correlations in real data

In [ ]:
data_kendall = list(stat_df["kendall"])[:num_of_days-1]
data_w_kendall = list(stat_df["w_kendall"])[:num_of_days-1]

# Popularity model

In [ ]:
print num_of_users, num_of_days

**TODO: fit powerlaw exponent on real data aggregated centrality values!!!**

In [ ]:
model = pm.PopularityModel(num_of_users, num_of_days)

### I. popularity of users

In [ ]:
ax = sns.distplot(model.U)

### II. daily variations

In [ ]:
ax = sns.distplot(model.alpha[:,0])

### III. calculate daily centrality scores (without Markov model)

In [ ]:
ax = sns.distplot(model.X[0,:])

### IV. Introducing Markov model without leaders

In [ ]:
X_act = model.get_centrality_with_markov(p, p_overlap)

### V. Introducing Markov model with leaders

In [ ]:
X_act_leaders = model.get_centrality_with_markov(p, p_overlap, lambda_=0.1)

In [ ]:
import scipy.stats as ss

def tiedrank(vector):
    return (len(vector) + 1) * np.ones(len(vector)) - ss.rankdata(vector)

def get_list_for_corr(M,day_idx):
    idx = day_idx
    day_one = np.ceil(M[idx,:])
    day_two = np.ceil(M[idx+1,:])

    ind_one=np.nonzero(day_one)[0];
    ind_two=np.nonzero(day_two)[0];
    ind=np.union1d(ind_one,ind_two)

    ranks_day_one=tiedrank(day_one[ind])
    ranks_day_two=tiedrank(day_two[ind])
    return ranks_day_one, ranks_day_two

In [ ]:
rank_list_0, rank_list_1 = get_list_for_corr(X_act_leaders,0)

In [ ]:
def findWKendall(rankX,rankY):
    n = len(rankX)
    denomX=0
    denomY=0
    denomXW=0
    denomYW=0
    num=0
    numW=0

    for i in range(n):
        for j in range(i+1,n):
            weightXY= 1/rankY[i]+1/rankY[j]
            weightX=1/rankX[i]+1/rankX[j];
            weightY=1/rankY[i]+1/rankY[j];
            termX=np.sign(rankX[i]-rankX[j]);
            termY=np.sign(rankY[i]-rankY[j]);
            denomX=denomX+(termX)**2;
            denomY=denomY+(termY)**2;
            denomXW=denomXW+(termX)**2*weightX;
            denomYW=denomYW+(termY)**2*weightY;
            num=num+termX*termY;
            numW=numW+termX*termY*weightXY;

    Kendall=num/math.sqrt(denomX*denomY);
    WKendall=numW/math.sqrt(denomXW*denomYW);
    return [Kendall, WKendall]

import multiprocessing
import itertools

def worker_init(rank_a,rank_b):
    global rankX
    global rankY
    rankX = rank_a
    rankY = rank_b

def worker(tuple_):
    i, j = tuple_[0], tuple_[1]
    weightXY= 1/rankY[i]+1/rankY[j]
    weightX=1/rankX[i]+1/rankX[j]
    weightY=1/rankY[i]+1/rankY[j]
    termX=np.sign(rankX[i]-rankX[j])
    termY=np.sign(rankY[i]-rankY[j])
    # denomX, denomY, denomXW, denomYW, num, numW
    return i, j, [termX**2, termY**2, termX**2*weightX, termY**2*weightY, termX*termY, termX*termY*weightXY]

def findWKendall2(rank_a,rank_b,num_proc=None):
    """parallel implementation"""
    size = len(rank_a)
    denomX = np.zeros((size,size))
    denomY = np.zeros((size,size))
    denomXW = np.zeros((size,size))
    denomYW = np.zeros((size,size))
    num = np.zeros((size,size))
    numW = np.zeros((size,size))
    
    pool = multiprocessing.Pool(processes=num_proc,initializer=worker_init, initargs=(rank_a,rank_b,))
    for i, j, val in pool.map(worker, itertools.combinations(range(size), 2)):
        denomX[i,j] = val[0] 
        denomY[i,j] = val[1] 
        denomXW[i,j] = val[2] 
        denomYW[i,j] = val[3] 
        num[i,j] = val[4] 
        numW[i,j] = val[5]
    Kendall=num.sum()/math.sqrt(denomX.sum()*denomY.sum());
    WKendall=numW.sum()/math.sqrt(denomXW.sum()*denomYW.sum());
    pool.close()
    return Kendall, WKendall

In [ ]:
import scipy.stats as stats

In [ ]:
%%time
print stats.kendalltau(rank_list_0, rank_list_1)

In [ ]:
%%time
findWKendall(rank_list_0, rank_list_1)

%%time
findWKendall2(rank_list_0, rank_list_1)

# Export centrality scores

In [ ]:
import os

def export_daily_scores(output_folder, M):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i in range(num_of_days):
        f = open(output_folder + '/centrality_scores_%i.txt' % i,'w')
        for j in range(num_of_users):
            if M[i,j] > 0.0:
                f.write('%i %f\n' % (j,M[i,j]))
        f.close()
    print 'Daily scores were exported to files.'                

In [ ]:
score_folder = "%s/popularity_scores" % experiment_folder

In [ ]:
export_daily_scores('%s/%s_pop_model/centrality_scores/' % (score_folder,dataset_id), model.X)

In [ ]:
export_daily_scores('%s/%s_pop_model_markov/centrality_scores/' % (score_folder,dataset_id), X_act)

In [ ]:
export_daily_scores('%s/%s_pop_model_leaders/centrality_scores/' % (score_folder,dataset_id), X_act_leaders)